In [4]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 31.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 33.8 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 31.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 33.8 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 41.3 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
import joblib
import boto3

In [19]:
# The digits dataset
digits = datasets.load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

joblib.dump(classifier, "mnist-svm.joblib")

['mnist-svm.joblib']

In [22]:
# MinIO configuration
minio_endpoint = "192.168.200.171:30900"  # Example: "localhost:9000"
access_key = "minio"               # Replace with your MinIO access key
secret_key = "minio123"               # Replace with your MinIO secret key
bucket_name = "openshift-ai"             # Replace with your bucket name
model_file_name = "mnist-svm.joblib"
folder_name = "minst-wb/"

In [23]:
# Initialize MinIO client
s3_client = boto3.client(
    's3',
    endpoint_url=f"http://{minio_endpoint}",  # Use "https://" if MinIO is configured for HTTPS
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)

# Upload the model file to the new folder
try:
    # Create a "folder" in the bucket by uploading an empty object with a trailing slash
    s3_client.put_object(Bucket=bucket_name, Key=folder_name)

    # Upload the model file into the folder
    with open(model_file_name, "rb") as model_file:
        s3_client.upload_fileobj(
            Fileobj=model_file,
            Bucket=bucket_name,
            Key=f"{folder_name}{model_file_name}"
        )
    print(f"Model '{model_file_name}' uploaded successfully to '{folder_name}' in bucket '{bucket_name}'.")
except FileNotFoundError:
    print(f"Error: File '{model_file_name}' not found.")
except NoCredentialsError:
    print("Error: MinIO credentials not available.")
except Exception as e:
    print(f"An error occurred: {e}")

Model 'mnist-svm.joblib' uploaded successfully to 'minst-wb/' in bucket 'openshift-ai'.
